Task 1
    The problem we have is that we have two videos: one with a single moving ball, that has an obstruction in its path where you can't see the ball move in the video, and other video with two moving balls and an obstructuion where both balls overlap in the same position in the video. We have to to make a detector to get the bounding boxes of the balls in both videos when they're on the screen and then use a kalman filter to predict the trajectory of each ball and then make those predictions shown on the video. 
    The kalman filter is an algorithm that predicts the variables of the next state of a system by calculating the previous state of the system by the relationships of each variable(state covariance matrix) in the state to each other with some adjustments for variables outside the system in the form of a control matrix and vector. There are three other parts to the kalman filter, with the next part being the uncertainty prediction, which is the amount of variance for each prediction representing the error range. This is calculated by the state covariance matrix, its transpose and the previous uncertainty as well as a possiobly noise matrix to added to them to account for noise in the prediction. Now this accounts for the prediction parts but kalman filter is an iterative method so there has to be a way to update the predicted values, in this case the other two parts of the kalman filter: the update step for the predicted state updates the predcited state by accouting for measurements in the system of the predicted variables in the state, now this can be done with a direct measurement or an indirect measurement by accounting for noise from the sensor as well in a state-to-measuremnt matrix. It also uses the kalman gain to calculate the update. The kalman gain is a value that tell us how much each estimate changes with the given measurement, it's calcualted by using the state-to-measurement matrix, stat covariance matrix, and the measurement noise covariance matrix, which helps with noise in indirect measurements. The second part of the update step updates the uncertainty prediction by using the prediction uncertainty, kalman gain, and measurement noise covariance matrix. We need to define a lot of the matrices used in kalman filter oursleves as well as the relationship between variables, measurement, and sensors and give an initial state and unvertainty to start with.
    So, I got the detector to work by making the video greyscale on each frame and highlighting the moving objects by making it movement show up as white. Then I limited the area to make the bounding box of the moving objects match the objects we want. This wasn't too much of a trouble as it highlighted the objects pretty well if they were on the screen and in the case of the two moving balls, when they overlapped the bounding box was over their overlapped space. I then got the positions, which for the single ball video was easy as there was only ever one bounding box for us to gather positons from, while for the two moving balls I got both of their positions back, so I then made a formula that got both objects values by looking at the relationship between each moving object and see if it was decreasin or increasing in each frame. This required a starting position for each object to be determined by me. After that I used all of those values returend from the bounding boxes as measurements for the kalman filter, the matricies in the kalman filter was easy to determine as the relationship between measureents and prediction was direct and the rlationship between the variables, each axis was it's own variable, in the state was direct as well. This ended up working well as the kalman filter's predictions lined up with the actual measurements.
    The problem happened after this step, when I tried to mark the prediction in the video. I got the frame numbers and all of the positions, if their are less positions than frames, it's either going to be the max value of min value of the frame since the balls are at the edge so I can fill that in. the problem is that I didn't know how to have a drawing showed on a frame by frame basis as it just shows all of the predictions at once on each frame. But it does show the path of the object with the kalman filter and not by detecing objects, as shown when they path beyond the overlap of the moving objects happen.

In [1]:
import cv2
import math
import numpy as np

Notes: ### = Code that can run, # = Explanation

Object Detection - Single Ball

In [2]:
cap = cv2.VideoCapture('C:/Users/poona/Downloads/ball.mp4')
od = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=150) #Object detector/Motion analysis
#history = How many previous frames are used for the background; The longer then more precise, but less precise if camera moves 
#varThreshold: Sqaured distance from pixel to sample; The lower the value the higher chance for false positives but more results
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #Frame count
print(length)

51


In [3]:
Coordinates = [] #For tracker, list of coordinates
while(cap.isOpened()):#Loop forever
    
    ret, frame = cap.read() #Knows if there is a frame and what the frame is
    
    if ret:   #ret is true if a frame is there, frame is the next frame in the video 
        
        ###height, width, _ = frame.shape
        ###print(height,width) #This was to get the area to cut down on when performing the motion analysis
        roi = frame[180:360,1:960] #Portion of the frame being used

        mask = od.apply(roi) #Applying the motion analysis to the frame: black if no movement, white if movement
        a, mask = cv2.threshold(mask,254,255,cv2.THRESH_BINARY) #Removes all non-white colors - Good for shadows
        #arguments: Frame, Threshold value, Max value, Function that replaces it(Binary function in this case)
        
        contours, a = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Boundary box
        #arguments for findContours: Image, Mode, Method/Algorithm that keeps the pixels(In this case all unique values only)
        #The mode gets all of the bounday box pixels

        for i in contours: #Goes by each frame
            area = cv2.contourArea(i) #Calculate area and remove small elements
            if area > 1400: 
                ###cv2.drawContours(roi,[i],-1,(0,255,0),2) #Draws boundary lines on the cropped frame
                #Arguments are: Frame, Contours, Contour index(For all put -1) , Color, Thickness
                
                x,y,h,w = cv2.boundingRect(i)#Draws a rectangle around the objects
                cv2.rectangle(roi,(x,y),(x+h,y+w),(0,255,0),3) #Draws a rectangle on cropped image
                #Arguments are: Image, Vertices 1, Verticies 2(The opposite side), Color, Line thickness
                Coordinates.append([x,y,h,w]) #For Tracker, adds the coordinates to the list
            
        
        cv2.imshow("Frame", frame) #Shows the frame from VideoCapture
        ###cv2.imshow("Mask",mask) #Shows frame after motion analysis was done to it
        ###cv2.imshow("ROI",roi) #Shows the frame after area was cropped out

        k = cv2.waitKey(200) #Time in milliseconds for each frame to pass
        if k == 27: #ESC key
            break #Ends loop
    else:
        break
cap.release() #Releases camera/video
cv2.destroyAllWindows() #Closes all open windows that showed frames

Object Tracker - Single Ball - Prediction with Kalman Filter

In [4]:
Coordarray = np.array(Coordinates) #Grabbing the Coordinates and making them into an array
xvals=np.zeros(31) #Empty array for x values
yvals=np.zeros(31) #Empty array for y values
hvals=np.zeros(31) #Empty array for y values
wvals=np.zeros(31) #Empty array for y values

for i in range(0,31): #Cycles through each placement in the x values array
    xvals[i]=Coordarray[i,0] #Updates it with the corresponding x value in the coordinates array

for i in range(0,31): #Cycles through each placement in the y values array
    yvals[i]=Coordarray[i,1] #Updates it with the corresponding y value in the coordinates array
    
for i in range(0,31): #Cycles through each placement in the h values array
    hvals[i]=Coordarray[i,2] #Updates it with the corresponding h value in the coordinates array
    
for i in range(0,31): #Cycles through each placement in the w values array
    wvals[i]=Coordarray[i,3] #Updates it with the corresponding w value in the coordinates array

In [5]:
#I did not need some of these, but was messing around with them so I just left them defined
#I believe 1's works for them because the measured values are waht we want and not some indirect measure
SCM=np.identity(1) #State Covariance Matrix
Q = np.ones((1,1)) #Process Noise Covariance Matrix
H = np.ones((1,1)) #Representation of Sensor
R = np.ones((1,1)) #Measurement Noise Covariance Matrix
zx=xvals #Measured values
zy=yvals #Measured values
zh=hvals #Measured values
zw=wvals #Measured values
ite=31 #Iterations
xpred=960 #Initial State
ypred=45 #Initial State
hpred=100
wpred=100
xtrack=np.zeros(ite) #Array to get all of the predictions
ytrack=np.zeros(ite)
htrack=np.zeros(ite)
wtrack=np.zeros(ite)
for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    xpred=SCM*xpred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*xpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    print(xpred) #Prints the predicted values ##Pretty close to the actual
    
    #Update Steps
    xupdate=xpred+K*(zx[i]-H*xpred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    xpred=xupdate #Updates the predicted value for the next iteration
    xtrack[i]=xupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    ypred=SCM*ypred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*xpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    ###print(ypred) #Prints the predicted values #Pretty close to the actual
    
    #Update Steps
    yupdate=ypred+K*(zy[i]-H*ypred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    ypred=yupdate #Updates the predicted value for the next iteration
    ytrack[i]=yupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    hpred=SCM*hpred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*hpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    ###print(hpred) #Prints the predicted values #Pretty close to the actual
    
    #Update Steps
    hupdate=hpred+K*(zh[i]-H*hpred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    hpred=hupdate #Updates the predicted value for the next iteration
    htrack[i]=hupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    wpred=SCM*wpred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*wpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    ###print(wpred) #Prints the predicted values #Pretty close to the actual
    
    #Update Steps
    wupdate=wpred+K*(zw[i]-H*wpred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    wpred=wupdate #Updates the predicted value for the next iteration
    wtrack[i]=wupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

[[960.]]
[[884.]]
[[889.5]]
[[835.25]]
[[846.125]]
[[765.5625]]
[[712.78125]]
[[674.890625]]
[[644.9453125]]
[[622.47265625]]
[[610.73632812]]
[[597.36816406]]
[[464.68408203]]
[[387.84204102]]
[[339.42102051]]
[[302.71051025]]
[[276.85525513]]
[[253.92762756]]
[[230.96381378]]
[[210.48190689]]
[[183.74095345]]
[[167.37047672]]
[[149.18523836]]
[[130.59261918]]
[[110.79630959]]
[[81.8981548]]
[[67.4490774]]
[[54.2245387]]
[[35.11226935]]
[[17.55613467]]
[[8.77806734]]


Object Tracker - Single Ball - Video

In [6]:
trialx=np.append(xtrack,np.zeros(20))
trialy=np.append(ytrack,np.zeros(20))
trialh=np.append(htrack,np.zeros(20))
trialw=np.append(wtrack,np.zeros(20))

In [7]:
cap = cv2.VideoCapture('C:/Users/poona/Downloads/ball.mp4')
od = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=150) #Object detector/Motion analysis
#history = How many previous frames are used for the background; The longer then more precise, but less precise if camera moves 
#varThreshold: Sqaured distance from pixel to sample; The lower the value the higher chance for false positives but more results

In [8]:
while(cap.isOpened()):#Loop forever
    
    ret, frame = cap.read() #Knows if there is a frame and what the frame is
    
    if ret:   #ret is true if a frame is there, frame is the next frame in the video 
        
        ###height, width, _ = frame.shape
        ###print(height,width) #This was to get the area to cut down on when performing the motion analysis
        roi = frame[180:360,1:960] #Portion of the frame being used

        mask = od.apply(roi) #Applying the motion analysis to the frame: black if no movement, white if movement
        a, mask = cv2.threshold(mask,254,255,cv2.THRESH_BINARY) #Removes all non-white colors - Good for shadows
        #arguments: Frame, Threshold value, Max value, Function that replaces it(Binary function in this case)
        
        contours, a = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Boundary box
        #arguments for findContours: Image, Mode, Method/Algorithm that keeps the pixels(In this case all unique values only)
        #The mode gets all of the bounday box pixels

        for i in contours: #Goes by each frame
            area = cv2.contourArea(i) #Calculate area and remove small elements
            if area > 1400: 
                ###cv2.drawContours(roi,[i],-1,(0,255,0),2) #Draws boundary lines on the cropped frame
                #Arguments are: Frame, Contours, Contour index(For all put -1) , Color, Thickness
                
                x,y,h,w = cv2.boundingRect(i)#Draws a rectangle around the objects
                ###cv2.rectangle(roi,(x,y),(x+h,y+w),(0,255,0),3) #Draws a rectangle on cropped image
                #cv2.circle(roi, (trialx[i], trialy[i]), radius=0, color=(0, 0, 255), thickness=-1)
                #Arguments are: Image, Vertices 1, Verticies 2(The opposite side), Color, Line thickness
        
        for i in range(0,51):
            cv2.rectangle(roi,(int(trialx[i]),int(trialy[i])),(int(trialx[i])+int(trialh[i]),int(trialy[i])+int(trialw[i])),(0,255,0),3)
        
        cv2.imshow("Frame", frame) #Shows the frame from VideoCapture
        ###cv2.imshow("Mask",mask) #Shows frame after motion analysis was done to it
        ###cv2.imshow("ROI",roi) #Shows the frame after area was cropped out

        k = cv2.waitKey(200) #Time in milliseconds for each frame to pass
        if k == 27: #ESC key
            break #Ends loop
    else:
        break
cap.release() #Releases camera/video
cv2.destroyAllWindows() #Closes all open windows that showed frames

Object Detection - Multiple Ball

In [9]:
cap= cv2.VideoCapture('C:/Users/poona/Downloads/objectTracking_examples_multiObject.avi') 
od = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=90) #Object detector/Motion analysis
#history = How many previous frames are used for the background; The longer then more precise, but less precise if camera moves 
#varThreshold: Sqaured distance from pixel to sample; The lower the value the higher chance for false positives but more results
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #Frame count
print(length)

41


In [10]:
MultiCoordinates = [] #For tracker, list of coordinates
while(cap.isOpened()):#Loop forever
    
    ret, frame = cap.read() #Knows if there is a frame and what the frame is
    
    if ret:   #ret is true if a frame is there, frame is the next frame in the video 
        
        height, width, _ = frame.shape
        ###print(height,width) #This was to get the area to cut down on when performing the motion analysis
        roi = frame[115:245,1:640] #Portion of the frame being used

        mask = od.apply(roi) #Applying the motion analysis to the frame: black if no movement, white if movement
        a, mask = cv2.threshold(mask,254,255,cv2.THRESH_BINARY) #Removes all non-white colors - Good for shadows
        #arguments: Frame, Threshold value, Max value, Function that replaces it(Binary function in this case)
        
        contours, a = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Boundary box
        #arguments for findContours: Image, Mode, Method/Algorithm that keeps the pixels(In this case all unique values only)
        #The mode gets all of the bounday box pixels

        for i in contours: #Goes by each frame
            area = cv2.contourArea(i) #Calculate area and remove small elements
            if area > 1400: 
                ###cv2.drawContours(roi,[i],-1,(0,255,0),2) #Draws boundary lines on the cropped frame
                #Arguments are: Frame, Contours, Contour index(For all put -1) , Color, Thickness
                
                x,y,h,w = cv2.boundingRect(i)#Draws a rectangle around the objects
                cv2.rectangle(roi,(x,y),(x+h,y+w),(0,255,0),3) #Draws a rectangle on cropped image
                #Arguments are: Image, Vertices 1, Verticies 2(The opposite side), Color, Line thickness
                MultiCoordinates.append([x,y,h,w]) #For Tracker, adds the coordinates to the list
        
        cv2.imshow("Frame", frame) #Shows the frame from VideoCapture
        ###cv2.imshow("Mask",mask) #Shows frame after motion analysis was done to it
        ###cv2.imshow("ROI",roi) #Shows the frame after area was cropped out

        k = cv2.waitKey(200) #Time in milliseconds for each frame to pass
        if k == 27: #ESC key
            break #Ends loop
    else:
        break
cap.release() #Releases camera/video
cv2.destroyAllWindows() #Closes all open windows that showed frames

Object Tracker - Multiple Ball - Prediction with Kalman Filter

In [11]:
MCoordarray=np.array(MultiCoordinates)
xfirst=np.zeros(52) #Empty vector to get a vector with the index but it wil be padded with zeros
xfirst[0]=566 #Initial value for the first object
initialx=xfirst[0] #Constant value of the first index in the vector
for i in range(0,52): #Checking for values
    if MCoordarray[i,0]<initialx:
         if (initialx-MCoordarray[i,0])<50:
            xfirst[i]=MCoordarray[i,0]
            initialx=xfirst[i] 
index = np.where(xfirst != 0)[0] #Index for first object

xfo=np.zeros(27) #First object positions
for i in range(0,27):
    xfo[i]=index[i]

allv=np.zeros(52) #Vector with all values
for i in range(0,52):
    allv[i]=i

xso= np.array(list(set(allv)-set(xfo))) #Second object positions

In [12]:
xfvals=np.zeros(27) #Empty array for x values
yfvals=np.zeros(27) #Empty array for y values
hfvals=np.zeros(27) #Empty array for y values
wfvals=np.zeros(27) #Empty array for y values

for i in range(0,27): #Cycles through each placement in the x values array
    xfvals[i]=MCoordarray[int(xfo[i]),0] #Updates it with the corresponding x value in the coordinates array

for i in range(0,27): #Cycles through each placement in the y values array
    yfvals[i]=MCoordarray[int(xfo[i]),1] #Updates it with the corresponding y value in the coordinates array
    
for i in range(0,27): #Cycles through each placement in the h values array
    hfvals[i]=MCoordarray[int(xfo[i]),2] #Updates it with the corresponding h value in the coordinates array
    
for i in range(0,27): #Cycles through each placement in the w values array
    wfvals[i]=MCoordarray[int(xfo[i]),3] #Updates it with the corresponding w value in the coordinates array
    
xsvals=np.zeros(25) #Empty array for x values
ysvals=np.zeros(25) #Empty array for y values
hsvals=np.zeros(25) #Empty array for y values
wsvals=np.zeros(27) #Empty array for y values

for i in range(0,25): #Cycles through each placement in the x values array
    xsvals[i]=MCoordarray[int(xso[i]),0] #Updates it with the corresponding x value in the coordinates array

for i in range(0,25): #Cycles through each placement in the y values array
    ysvals[i]=MCoordarray[int(xso[i]),1] #Updates it with the corresponding y value in the coordinates array
    
for i in range(0,25): #Cycles through each placement in the h values array
    hsvals[i]=MCoordarray[int(xso[i]),2] #Updates it with the corresponding h value in the coordinates array
    
for i in range(0,25): #Cycles through each placement in the w values array
    wsvals[i]=MCoordarray[int(xso[i]),3] #Updates it with the corresponding w value in the coordinates array

In [13]:
#First object predictions
#I did not need some of these, but was messing around with them so I just left them defined
#I believe 1's works for them because the measured values are waht we want and not some indirect measure
SCM=np.identity(1) #State Covariance Matrix
Q = np.ones((1,1)) #Process Noise Covariance Matrix
H = np.ones((1,1)) #Representation of Sensor
R = np.ones((1,1)) #Measurement Noise Covariance Matrix
zx=xfvals #Measured values
zy=yfvals #Measured values
zh=hfvals #Measured values
zw=wfvals #Measured values
ite=27 #Iterations
xpred=600 #Initial State
ypred=10 #Initial State
hpred=100
wpred=100
xftrack=np.zeros(ite) #Array to get all of the predictions
yftrack=np.zeros(ite)
hftrack=np.zeros(ite)
wftrack=np.zeros(ite)
for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    xpred=SCM*xpred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*xpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    print(xpred) #Prints the predicted values ##Pretty close to the actual
    
    #Update Steps
    xupdate=xpred+K*(zx[i]-H*xpred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    xpred=xupdate #Updates the predicted value for the next iteration
    xftrack[i]=xupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    ypred=SCM*ypred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*xpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    ###print(ypred) #Prints the predicted values #Pretty close to the actual
    
    #Update Steps
    yupdate=ypred+K*(zy[i]-H*ypred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    ypred=yupdate #Updates the predicted value for the next iteration
    yftrack[i]=yupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    hpred=SCM*hpred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*hpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    ###print(hpred) #Prints the predicted values #Pretty close to the actual
    
    #Update Steps
    hupdate=hpred+K*(zh[i]-H*hpred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    hpred=hupdate #Updates the predicted value for the next iteration
    hftrack[i]=hupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    wpred=SCM*wpred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*wpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    ###print(wpred) #Prints the predicted values #Pretty close to the actual
    
    #Update Steps
    wupdate=wpred+K*(zw[i]-H*wpred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    wpred=wupdate #Updates the predicted value for the next iteration
    wftrack[i]=wupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

[[600.]]
[[583.]]
[[562.]]
[[538.5]]
[[516.75]]
[[494.375]]
[[472.1875]]
[[450.59375]]
[[428.796875]]
[[407.3984375]]
[[386.19921875]]
[[365.59960938]]
[[345.29980469]]
[[325.14990234]]
[[291.57495117]]
[[269.28747559]]
[[240.14373779]]
[[216.0718689]]
[[195.03593445]]
[[175.51796722]]
[[156.25898361]]
[[137.62949181]]
[[119.3147459]]
[[101.15737295]]
[[83.07868648]]
[[64.53934324]]
[[46.26967162]]


In [14]:
#Second object predictions
#I did not need some of these, but was messing around with them so I just left them defined
#I believe 1's works for them because the measured values are waht we want and not some indirect measure
SCM=np.identity(1) #State Covariance Matrix
Q = np.ones((1,1)) #Process Noise Covariance Matrix
H = np.ones((1,1)) #Representation of Sensor
R = np.ones((1,1)) #Measurement Noise Covariance Matrix
zx=xvals[::-1] #Measured values - Had to reverse them since the index was from the other side of the frame
zy=yvals[::-1] #Measured values
zh=hvals[::-1] #Measured values
zw=wvals[::-1] #Measured values
ite=25 #Iterations
xpred=0 #Initial State
ypred=10 #Initial State
hpred=100
wpred=100
xstrack=np.zeros(ite) #Array to get all of the predictions
ystrack=np.zeros(ite)
hstrack=np.zeros(ite)
wstrack=np.zeros(ite)
for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    xpred=SCM*xpred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*xpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    print(xpred) #Prints the predicted values ##Pretty close to the actual
    
    #Update Steps
    xupdate=xpred+K*(zx[i]-H*xpred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    xpred=xupdate #Updates the predicted value for the next iteration
    xstrack[i]=xupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    ypred=SCM*ypred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*xpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    ###print(ypred) #Prints the predicted values #Pretty close to the actual
    
    #Update Steps
    yupdate=ypred+K*(zy[i]-H*ypred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    ypred=yupdate #Updates the predicted value for the next iteration
    ystrack[i]=yupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    hpred=SCM*hpred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*hpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    ###print(hpred) #Prints the predicted values #Pretty close to the actual
    
    #Update Steps
    hupdate=hpred+K*(zh[i]-H*hpred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    hpred=hupdate #Updates the predicted value for the next iteration
    hstrack[i]=hupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

for i in range(0,ite): #Cycles through number of iterations
    
    #Kalman Gain
    K=(SCM*H.T)/(H*SCM*H.T+R) #Relationship between prediction and measurement, only new variable is the measurement noise covariance matrix

    #Prediction Steps
    wpred=SCM*wpred #Prediction of the value by multiplying the previous or initial value by the state
    #No control matrix or vecotor was added for the state prediction
    
    Ppred=SCM*wpred*SCM.T + Q #Covariance prediction
    #Calculated by multiplying the state covariance matrix and its transpose by the predicted state value
    
    ###print(wpred) #Prints the predicted values #Pretty close to the actual
    
    #Update Steps
    wupdate=wpred+K*(zw[i]-H*wpred) #Updates the predicted state
    #Accounts for a measured value,a wight on the current estimate called the Kalman Gain, 
    #and the sensor measurement represenation
    
    wpred=wupdate #Updates the predicted value for the next iteration
    wstrack[i]=wupdate #Updates predicted values to an array
    Pupdate=SCM-K*H*SCM #Covariance prediction based on the Kalman Gain, sensor reprsentation, and state covariance matrix
    Ppred=Pupdate #Updates the predicted covariance

[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[8.]]
[[24.5]]
[[38.75]]
[[45.875]]
[[68.4375]]
[[90.21875]]
[[110.609375]]
[[130.8046875]]
[[143.90234375]]
[[166.95117188]]
[[187.47558594]]
[[209.23779297]]
[[230.11889648]]
[[248.05944824]]
[[269.52972412]]
[[290.26486206]]
[[311.13243103]]
[[447.56621552]]
[[523.28310776]]
[[561.64155388]]
[[588.32077694]]


Object Tracker - Multiple Ball - Video - Object 1

In [15]:
trialfx=np.append(xftrack,np.zeros(14))
trialfy=np.append(yftrack,np.zeros(14))
trialfh=np.append(hftrack,np.zeros(14))
trialfw=np.append(wftrack,np.zeros(14))

In [16]:
cap= cv2.VideoCapture('C:/Users/poona/Downloads/objectTracking_examples_multiObject.avi') 
od = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=150) #Object detector/Motion analysis
#history = How many previous frames are used for the background; The longer then more precise, but less precise if camera moves 
#varThreshold: Sqaured distance from pixel to sample; The lower the value the higher chance for false positives but more results

In [17]:
while(cap.isOpened()):#Loop forever
    
    ret, frame = cap.read() #Knows if there is a frame and what the frame is
    
    if ret:   #ret is true if a frame is there, frame is the next frame in the video 
        
        ###height, width, _ = frame.shape
        ###print(height,width) #This was to get the area to cut down on when performing the motion analysis
        roi = frame[115:245,1:640] #Portion of the frame being used

        mask = od.apply(roi) #Applying the motion analysis to the frame: black if no movement, white if movement
        a, mask = cv2.threshold(mask,254,255,cv2.THRESH_BINARY) #Removes all non-white colors - Good for shadows
        #arguments: Frame, Threshold value, Max value, Function that replaces it(Binary function in this case)
        
        contours, a = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Boundary box
        #arguments for findContours: Image, Mode, Method/Algorithm that keeps the pixels(In this case all unique values only)
        #The mode gets all of the bounday box pixels

        for i in contours: #Goes by each frame
            area = cv2.contourArea(i) #Calculate area and remove small elements
            if area > 1400: 
                ###cv2.drawContours(roi,[i],-1,(0,255,0),2) #Draws boundary lines on the cropped frame
                #Arguments are: Frame, Contours, Contour index(For all put -1) , Color, Thickness
                
                x,y,h,w = cv2.boundingRect(i)#Draws a rectangle around the objects
                ###cv2.rectangle(roi,(x,y),(x+h,y+w),(0,255,0),3) #Draws a rectangle on cropped image
                #cv2.circle(roi, (trialx[i], trialy[i]), radius=0, color=(0, 0, 255), thickness=-1)
                #Arguments are: Image, Vertices 1, Verticies 2(The opposite side), Color, Line thickness
        
        for i in range(0,41):
            cv2.rectangle(roi,(int(trialfx[i]),int(trialfy[i])),(int(trialfx[i])+int(trialfh[i]),int(trialfy[i])+int(trialfw[i])),(0,255,0),3)
        
        cv2.imshow("Frame", frame) #Shows the frame from VideoCapture
        ###cv2.imshow("Mask",mask) #Shows frame after motion analysis was done to it
        ###cv2.imshow("ROI",roi) #Shows the frame after area was cropped out

        k = cv2.waitKey(200) #Time in milliseconds for each frame to pass
        if k == 27: #ESC key
            break #Ends loop
    else:
        break
cap.release() #Releases camera/video
cv2.destroyAllWindows() #Closes all open windows that showed frames

Object Tracker - Multiple Ball - Video - Object 2

In [18]:
trialsx=np.append(xstrack,np.zeros(16))
trialsy=np.append(ystrack,np.zeros(16))
trialsh=np.append(hstrack,np.zeros(16))
trialsw=np.append(wstrack,np.zeros(16))

In [19]:
cap= cv2.VideoCapture('C:/Users/poona/Downloads/objectTracking_examples_multiObject.avi') 
od = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=150) #Object detector/Motion analysis
#history = How many previous frames are used for the background; The longer then more precise, but less precise if camera moves 
#varThreshold: Sqaured distance from pixel to sample; The lower the value the higher chance for false positives but more results

In [20]:
while(cap.isOpened()):#Loop forever
    
    ret, frame = cap.read() #Knows if there is a frame and what the frame is
    
    if ret:   #ret is true if a frame is there, frame is the next frame in the video 
        
        ###height, width, _ = frame.shape
        ###print(height,width) #This was to get the area to cut down on when performing the motion analysis
        roi = frame[115:245,1:640] #Portion of the frame being used

        mask = od.apply(roi) #Applying the motion analysis to the frame: black if no movement, white if movement
        a, mask = cv2.threshold(mask,254,255,cv2.THRESH_BINARY) #Removes all non-white colors - Good for shadows
        #arguments: Frame, Threshold value, Max value, Function that replaces it(Binary function in this case)
        
        contours, a = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Boundary box
        #arguments for findContours: Image, Mode, Method/Algorithm that keeps the pixels(In this case all unique values only)
        #The mode gets all of the bounday box pixels

        for i in contours: #Goes by each frame
            area = cv2.contourArea(i) #Calculate area and remove small elements
            if area > 1400: 
                ###cv2.drawContours(roi,[i],-1,(0,255,0),2) #Draws boundary lines on the cropped frame
                #Arguments are: Frame, Contours, Contour index(For all put -1) , Color, Thickness
                
                x,y,h,w = cv2.boundingRect(i)#Draws a rectangle around the objects
                ###cv2.rectangle(roi,(x,y),(x+h,y+w),(0,255,0),3) #Draws a rectangle on cropped image
                #cv2.circle(roi, (trialx[i], trialy[i]), radius=0, color=(0, 0, 255), thickness=-1)
                #Arguments are: Image, Vertices 1, Verticies 2(The opposite side), Color, Line thickness
        
        for i in range(0,41):
            cv2.rectangle(roi,(int(trialsx[i]),int(trialsy[i])),(int(trialsx[i])+int(trialsh[i]),int(trialsy[i])+int(trialsw[i])),(0,255,0),3)
        
        cv2.imshow("Frame", frame) #Shows the frame from VideoCapture
        ###cv2.imshow("Mask",mask) #Shows frame after motion analysis was done to it
        ###cv2.imshow("ROI",roi) #Shows the frame after area was cropped out

        k = cv2.waitKey(200) #Time in milliseconds for each frame to pass
        if k == 27: #ESC key
            break #Ends loop
    else:
        break
cap.release() #Releases camera/video
cv2.destroyAllWindows() #Closes all open windows that showed frames

Object Tracker - Multiple Ball - Video - All Objects

In [21]:
cap= cv2.VideoCapture('C:/Users/poona/Downloads/objectTracking_examples_multiObject.avi') 
od = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=150) #Object detector/Motion analysis
#history = How many previous frames are used for the background; The longer then more precise, but less precise if camera moves 
#varThreshold: Sqaured distance from pixel to sample; The lower the value the higher chance for false positives but more results

In [22]:
while(cap.isOpened()):#Loop forever
    
    ret, frame = cap.read() #Knows if there is a frame and what the frame is
    
    if ret:   #ret is true if a frame is there, frame is the next frame in the video 
        
        ###height, width, _ = frame.shape
        ###print(height,width) #This was to get the area to cut down on when performing the motion analysis
        roi = frame[115:245,1:640] #Portion of the frame being used

        mask = od.apply(roi) #Applying the motion analysis to the frame: black if no movement, white if movement
        a, mask = cv2.threshold(mask,254,255,cv2.THRESH_BINARY) #Removes all non-white colors - Good for shadows
        #arguments: Frame, Threshold value, Max value, Function that replaces it(Binary function in this case)
        
        contours, a = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Boundary box
        #arguments for findContours: Image, Mode, Method/Algorithm that keeps the pixels(In this case all unique values only)
        #The mode gets all of the bounday box pixels

        for i in contours: #Goes by each frame
            area = cv2.contourArea(i) #Calculate area and remove small elements
            if area > 1400: 
                ###cv2.drawContours(roi,[i],-1,(0,255,0),2) #Draws boundary lines on the cropped frame
                #Arguments are: Frame, Contours, Contour index(For all put -1) , Color, Thickness
                
                x,y,h,w = cv2.boundingRect(i)#Draws a rectangle around the objects
                ###cv2.rectangle(roi,(x,y),(x+h,y+w),(0,255,0),3) #Draws a rectangle on cropped image
                #cv2.circle(roi, (trialx[i], trialy[i]), radius=0, color=(0, 0, 255), thickness=-1)
                #Arguments are: Image, Vertices 1, Verticies 2(The opposite side), Color, Line thickness
        
        for i in range(0,41):
            cv2.rectangle(roi,(int(trialfx[i]),int(trialfy[i])),(int(trialfx[i])+int(trialfh[i]),int(trialfy[i])+int(trialfw[i])),(0,255,0),3)
            cv2.rectangle(roi,(int(trialsx[i]),int(trialsy[i])),(int(trialsx[i])+int(trialsh[i]),int(trialsy[i])+int(trialsw[i])),(255,0,0),3)
            
        cv2.imshow("Frame", frame) #Shows the frame from VideoCapture
        ###cv2.imshow("Mask",mask) #Shows frame after motion analysis was done to it
        ###cv2.imshow("ROI",roi) #Shows the frame after area was cropped out

        k = cv2.waitKey(200) #Time in milliseconds for each frame to pass
        if k == 27: #ESC key
            break #Ends loop
    else:
        break
cap.release() #Releases camera/video
cv2.destroyAllWindows() #Closes all open windows that showed frames